In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from bert_serving.client import BertClient
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from nltk.corpus import stopwords
from gensim.models import Word2Vec
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
import gensim
from gensim import corpora

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\603766\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv('../data/metadata.csv')

In [3]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [4]:
abstracts=df['abstract']

In [5]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
str_abstract=[str(abstract) for abstract in df['title']]
abstract_array = [clean(abstract).split() for abstract in str_abstract] 

In [6]:
sentences = abstract_array
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = [sentence + " [SEP] [CLS]" for sentence in new_sent]


In [7]:
tokenizer_1 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts_1 = [tokenizer_1.tokenize(sent) for sent in sentences]
print(tokenized_texts_1[0])

['[', "'", 'si', '##ann', "'", ',', "'", 'strain', "'", ',', "'", 'identification', "'", ',', "'", 'alignment', "'", ',', "'", 'near', "'", ',', "'", 'neighbor', "'", ']', '[SEP]', '[CLS]']


In [ ]:
#use kmeans clustering, create and unsupervised model
#then input documents into it
#sort them into different data frames based on their clutser
# make a dictionary and use topic modeling to identify a pattern for each cluster 
#Do this by identifying the frequency for tokens and comparing it back to 
#the goal is to idenfiy where our research efforts have been primarily focused 
# and where most topics remain underresearched 

In [8]:
MAX_LEN = 128

In [9]:
input_ids_1 = [tokenizer_1.convert_tokens_to_ids(x) for x in tokenized_texts_1]

In [10]:
#create dict between ids and tokenized texts
#see if you can use this in topic modeling

In [11]:
input_ids_1 = pad_sequences(input_ids_1, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [12]:
input_ids_1

array([[ 1031,  1005,  9033, ...,     0,     0,     0],
       [ 1031,  1005, 13589, ...,     0,     0,     0],
       [ 1031,  1005, 24558, ...,     0,     0,     0],
       ...,
       [ 1031,  1005,  2224, ...,     0,     0,     0],
       [ 1031,  1005,  4807, ...,     0,     0,     0],
       [ 1031,  1005,  6575, ...,     0,     0,     0]])

In [14]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(input_ids_1)

# Run LDA
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(input_ids_1)


In [15]:
lda.transform(input_ids_1)

array([[5.96145525e-02, 2.39774996e-01, 1.73616539e-01, ...,
        1.17796257e-06, 1.17794131e-06, 2.10164570e-01],
       [1.10961756e-01, 1.22384118e-01, 3.98740022e-01, ...,
        8.33179070e-07, 8.33162759e-07, 1.11008163e-01],
       [6.08252602e-04, 9.40531867e-03, 2.51375597e-01, ...,
        4.22145422e-01, 9.95358886e-02, 1.62639079e-02],
       ...,
       [3.45735494e-06, 8.65961352e-01, 3.45736616e-06, ...,
        3.45726447e-06, 3.45723884e-06, 1.29110989e-01],
       [4.21683745e-02, 5.07765620e-02, 7.05272269e-02, ...,
        9.28895734e-07, 9.28872530e-07, 3.26024845e-02],
       [8.25596292e-03, 1.47579976e-02, 3.74256349e-02, ...,
        2.08625725e-01, 3.26193571e-01, 1.62289743e-02]])

In [16]:
nmf.transform(input_ids_1)

array([[3.82865236e+00, 1.96608269e+00, 0.00000000e+00, ...,
        1.44153215e-01, 0.00000000e+00, 0.00000000e+00],
       [5.54482913e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.33686438e+00, 0.00000000e+00, 0.00000000e+00],
       [1.01352992e+01, 8.36611611e-01, 1.70106077e+01, ...,
        1.25529743e+00, 4.84132567e+00, 0.00000000e+00],
       ...,
       [9.31022787e-01, 1.08964680e-02, 0.00000000e+00, ...,
        1.94618908e-01, 1.04497700e-03, 2.13075369e-03],
       [1.43291463e+00, 2.30576839e+00, 0.00000000e+00, ...,
        8.43804519e+00, 0.00000000e+00, 1.89652714e+01],
       [2.56263129e+00, 3.50121499e+00, 1.62564640e+01, ...,
        2.02994151e+00, 5.47762619e-01, 4.46667307e+00]])

In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(input_ids_1)


In [18]:
kmeans.labels_

array([15, 12,  2, ..., 18,  7, 10])

In [19]:
kmeans.cluster_centers_

array([[1.03100000e+03, 1.00500000e+03, 9.50274466e+03, ...,
        1.57113080e+00, 1.26258468e+01, 7.49661282e+00],
       [1.03100000e+03, 1.00500000e+03, 8.19159623e+03, ...,
        1.90325761e+01, 8.61551650e-01, 8.75696528e-01],
       [1.03100000e+03, 1.00500000e+03, 8.93482792e+03, ...,
        6.04693141e-01, 4.14380265e+00, 7.15403129e+00],
       ...,
       [1.03100000e+03, 1.00500000e+03, 9.34227731e+03, ...,
        5.01246883e-01, 5.07231920e-01, 5.01246883e-01],
       [1.03100000e+03, 1.00500487e+03, 4.88401756e+03, ...,
        1.74691465e-01, 1.75560577e-01, 1.74691465e-01],
       [1.03100000e+03, 1.00500000e+03, 8.81545695e+03, ...,
        1.76814856e+00, 1.91896455e+00, 1.13393360e+00]])

In [20]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = [i for i in doc.lower().split() if i not in stop]
    punc_free = [ch for ch in stop_free if ch not in exclude]
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free)
    return normalized

 

In [21]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
#dct=Dictionary()
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(abstract_array)
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(abstract) for abstract in abstract_array]

In [22]:
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

KeyboardInterrupt: 

In [ ]:
print(ldamodel.print_topics(num_topics=10, num_words=3))

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
model = Word2Vec(doc_clean, min_count=1)
# fit a 2d PCA model to the vectors
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()